In [6]:
import numpy as np
if not hasattr(np, 'int'):
    np.int=int
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from unioncom import UnionCom
import os

In [13]:


intersect_adni = pd.read_csv("/nfs/turbo/umms-lgarmire/home/yhdu/Bowei_NAS/EFIGA/intersect_adni_metab.csv",index_col=0)  # ROSMAP GE
intersect_rosmap_mci = pd.read_csv("/nfs/turbo/umms-lgarmire/home/yhdu/Bowei_NAS/EFIGA/Review/Unioncome_Review/EFIGA_with_labels.csv", index_col=0)  # ADNI GE
intersect_rosmap_mci = intersect_rosmap_mci.iloc[:,:-1]

adni_mrna_merge = pd.read_csv("/nfs/turbo/umms-lgarmire/home/yhdu/Bowei_NAS/EFIGA/adni_mrna_merge.csv",index_col=0)


conditions = [
    (adni_mrna_merge['subtype'] == "EMCI1"),
    (adni_mrna_merge['subtype'] == "EMCI2") ,
    (adni_mrna_merge['subtype'] == "LMCI1") ,
    (adni_mrna_merge['subtype'] == "LMCI2")
    ]

# create a list of the values we want to assign for each condition
values = ['EMCI', 'EMCI', 'LMCI', 'LMCI']

# create a new column and use np.select to assign values to it using our lists as arguments
adni_mrna_merge['subtype_big'] = np.select(conditions, values)

adni_mrna_merge = adni_mrna_merge.sort_values(by=['Unnamed: 0'])
labels = adni_mrna_merge['subtype_big']

intersect_adni = intersect_adni.sort_index()
data1 = intersect_adni.to_numpy()
data2 = intersect_rosmap_mci.to_numpy()

#normalization
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
ss_scaler = preprocessing.StandardScaler()


norm_data1 = min_max_scaler.fit_transform(data1) ### adni
norm_data2 = min_max_scaler.fit_transform(data2) # ROSMAP

# Initialize UnionCom

uc = UnionCom.UnionCom(output_dim=11,epoch_pd=2000,epoch_DNN=0,distance_mode='geodesic')

# fit UnionCom

#computer and extracts the match results from the cor_pairs.
integrated_data = uc.fit_transform(dataset=[norm_data1,norm_data2]) 
part_1 = integrated_data[0]
part_2 = integrated_data[1]

# Get correlation pairs
cor_pairs = uc.match(dataset=[norm_data1,norm_data2])
norm_pat_match = cor_pairs[0]

def get_indices(lst, targets):
    indices = []
    for target in targets:
        if target in lst:
            indices.append(lst.index(target))
    return indices

#List of indices where the label is 'EMCI'
EMCI_ID = list(np.where(np.isin(labels,'EMCI'))[0])
LMCI_ID = list(np.where(np.isin(labels,'LMCI'))[0])
clus_in = EMCI_ID

# Match the EMCI labels 
matched = []   


##99.5% match EMCI-ADNI

thre = 0.95


for index in clus_in:
# extracting the correlation or similarity scores for correlation pairs from EMCI ID
    match_vec = norm_pat_match[index]
# similarity scores in match_vec are greater than a threshold value
    matched_index = np.where(match_vec > thre*max(match_vec))
    matched.append(matched_index[0])
# This list contains all indices that were identified as highly similar across all iterations
    flat_list = [item for sublist in matched for item in sublist]
#sort the list
new_flat_list = list(set(sorted(flat_list)))

UnionCom_predict =  ['LMCI'] * 717
for index in new_flat_list:
    UnionCom_predict[index] = 'EMCI'
    
intersect_rosmap_mci['labels_matching_UC_big'] = UnionCom_predict
intersect_rosmap_mci.to_csv('/nfs/turbo/umms-lgarmire/home/yhdu/Bowei_NAS/EFIGA/Review/Unioncome_Review/95.csv')

use random seed: 666
Shape of Raw data
Dataset 0: (401, 54)
Dataset 1: (717, 54)
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
use device: cpu
epoch:[100/2000] err:5.3921 alpha:6.7115
epoch:[200/2000] err:0.7224 alpha:0.8992
epoch:[300/2000] err:0.2784 alpha:0.3466
epoch:[400/2000] err:0.3140 alpha:0.3917
epoch:[500/2000] err:0.3470 alpha:0.4335
epoch:[600/2000] err:0.3781 alpha:0.4732
epoch:[700/2000] err:0.4071 alpha:0.5106
epoch:[800/2000] err:0.4340 alpha:0.5457
epoch:[900/2000] err:0.4587 alpha:0.5785
epoch:[1000/2000] err:0.4814 alpha:0.6090
epoch:[1100/2000] err:0.5019 alpha:0.6374
epoch:[1200/2000] err:0.5205 alpha:0.6636
epoch:[1300/2000] err:0.5371 alpha:0.6878
epoch:[1400/2000] err:0.5520 alpha:0.7102
epoch:[1500/2000] err:0.5651 alpha:0.7308
epoch:[1600/2000] err:0.5766 alpha:0.7497
epoch:[1700/2000] err:0.5867 alpha:0.7671
epoch:[1800/2000] err:0.5955 alpha:0.7831
epoch:[1900/2000] err:0.6032 alpha:0.7977
epoch:[2000/2000] err:0.6098

In [7]:
distance_modes = [
    'geodesic', 'l1'
]

for distance_mode in distance_modes:
    print(f"Running with distance mode: {distance_mode}")

    intersect_adni = pd.read_csv("/nfs/dcmb-lgarmire/boweil/EFIGA/DE_label/DE_ADNI_QN.csv", index_col=0)
    intersect_adni['Label'] = intersect_adni['Label'].replace({
        'subtype1': 'LMCI1',
        'subtype2': 'LMCI2'
    })
    LMCI_conditions = intersect_adni['Label'].isin(['LMCI1', 'LMCI2'])
    intersect_adni_LMCI = intersect_adni[LMCI_conditions]
    labels_ADNI = intersect_adni_LMCI['Label']
    intersect_adni_LMCI.drop(columns=['Label'], inplace=True)

    intersect_EFIGA_mci = pd.read_csv("/nfs/dcmb-lgarmire/boweil/EFIGA/DE_label/DE_EFIGA_QN.csv", index_col=0)

    EFIGA_LMCI_samples = intersect_EFIGA_mci.loc[
        intersect_EFIGA_mci['big label'] == 'LMCI'
    ]
    data2_LMCI = EFIGA_LMCI_samples.to_numpy()
    data2_LMCI = data2_LMCI[:, 1:]

    min_max_scaler = MinMaxScaler()
    norm_data1 = min_max_scaler.fit_transform(intersect_adni_LMCI.to_numpy())
    norm_data2 = min_max_scaler.fit_transform(data2_LMCI)

    uc = UnionCom.UnionCom(output_dim=11, epoch_pd=2000, epoch_DNN=0, distance_mode=distance_mode)
    integrated_data = uc.fit_transform(dataset=[norm_data1, norm_data2])
    cor_pairs = uc.match(dataset=[norm_data1, norm_data2])
    norm_pat_match = cor_pairs[0]

    print(f"norm_data1 shape: {norm_data1.shape}")
    print(f"norm_data2 shape: {norm_data2.shape}")
    print(f"norm_pat_match shape: {norm_pat_match.shape}")

    LMCI1_ID = list(np.where(np.isin(labels_ADNI, 'LMCI1'))[0])
    LMCI2_ID = list(np.where(np.isin(labels_ADNI, 'LMCI2'))[0])
    print(f"LMCI1_ID length: {len(LMCI1_ID)}")
    print(f"LMCI2_ID length: {len(LMCI2_ID)}")

    matched_LMCI1, matched_LMCI2 = [], []
    thre = 0.995
    ties = 0

    for index in range(norm_pat_match.shape[0]):
        match_vec = norm_pat_match[index]
        if max(LMCI1_ID) >= len(match_vec) or max(LMCI2_ID) >= len(match_vec):
            continue

        max_score_LMCI1 = max(match_vec[LMCI1_ID])
        max_score_LMCI2 = max(match_vec[LMCI2_ID])

        if abs(max_score_LMCI1 - max_score_LMCI2) < 1e-6:
            ties += 1
            print(f"Tie detected for sample index {index}.")

        if max_score_LMCI1 >= max_score_LMCI2:
            matched_LMCI1.append(index)
        else:
            matched_LMCI2.append(index)

    print(f"Number of matched LMCI1 samples: {len(matched_LMCI1)}")
    print(f"Number of matched LMCI2 samples: {len(matched_LMCI2)}")
    print(f"Number of ties detected: {ties}")

    flat_list_LMCI1 = list(set(sorted(matched_LMCI1)))
    flat_list_LMCI2 = list(set(sorted(matched_LMCI2)))

    intersect_EFIGA_mci['labels_matching_LMCI'] = 'Unmatched'
    efiga_lmci_indices = intersect_EFIGA_mci.loc[intersect_EFIGA_mci['big label'] == 'LMCI'].index

    for i, idx in enumerate(efiga_lmci_indices):
        if i in flat_list_LMCI1:
            intersect_EFIGA_mci.at[idx, 'labels_matching_LMCI'] = 'LMCI1'
        elif i in flat_list_LMCI2:
            intersect_EFIGA_mci.at[idx, 'labels_matching_LMCI'] = 'LMCI2'

    unmatched_indices = intersect_EFIGA_mci.loc[
        (intersect_EFIGA_mci['big label'] == 'LMCI') & 
        (intersect_EFIGA_mci['labels_matching_LMCI'] == 'Unmatched')
    ].index

    for idx in unmatched_indices:
        col_idx = list(efiga_lmci_indices).index(idx)
        similarity_scores = norm_pat_match[:, col_idx]
        highest_similarity_index = np.argmax(similarity_scores)
        highest_similarity_label = labels_ADNI.iloc[highest_similarity_index]
        intersect_EFIGA_mci.at[idx, 'labels_matching_LMCI'] = highest_similarity_label

    final_unmatched_count = intersect_EFIGA_mci.loc[
        (intersect_EFIGA_mci['big label'] == 'LMCI') & 
        (intersect_EFIGA_mci['labels_matching_LMCI'] == 'Unmatched')
    ].shape[0]

    print(f"Remaining unmatched LMCI samples: {final_unmatched_count}")
    print(f"Number of ties detected and resolved: {ties}")

    output_file = f'/nfs/turbo/umms-lgarmire/home/yhdu/Bowei_NAS/EFIGA/Review/Unioncome_Review/LMCI_mis_{distance_mode}.csv'
    intersect_EFIGA_mci.to_csv(output_file)

    print(f"The inferred labels have been saved to '{output_file}'.")
    

Running with distance mode: geodesic
use random seed: 666
Shape of Raw data
Dataset 0: (200, 11)
Dataset 1: (51, 11)
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
use device: cpu


/tmp/ipykernel_3063453/3541338624.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersect_adni_LMCI.drop(columns=['Label'], inplace=True)


epoch:[100/2000] err:0.1018 alpha:0.1760
epoch:[200/2000] err:0.0971 alpha:0.1680
epoch:[300/2000] err:0.0975 alpha:0.1687
epoch:[400/2000] err:0.0978 alpha:0.1694
epoch:[500/2000] err:0.0981 alpha:0.1699
epoch:[600/2000] err:0.0983 alpha:0.1704
epoch:[700/2000] err:0.0985 alpha:0.1709
epoch:[800/2000] err:0.0986 alpha:0.1713
epoch:[900/2000] err:0.0988 alpha:0.1717
epoch:[1000/2000] err:0.0989 alpha:0.1720
epoch:[1100/2000] err:0.0989 alpha:0.1724
epoch:[1200/2000] err:0.0990 alpha:0.1726
epoch:[1300/2000] err:0.0990 alpha:0.1729
epoch:[1400/2000] err:0.0990 alpha:0.1732
epoch:[1500/2000] err:0.0990 alpha:0.1734
epoch:[1600/2000] err:0.0990 alpha:0.1736
epoch:[1700/2000] err:0.0989 alpha:0.1738
epoch:[1800/2000] err:0.0989 alpha:0.1740
epoch:[1900/2000] err:0.0988 alpha:0.1742
epoch:[2000/2000] err:0.0987 alpha:0.1743
Finished Matching!
---------------------------------
Begin finding the embedded space
Done
---------------------------------
unionCom Done!
time: 0.0796213150024414 seco

/tmp/ipykernel_3063453/3541338624.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersect_adni_LMCI.drop(columns=['Label'], inplace=True)


epoch:[200/2000] err:0.1523 alpha:0.1770
epoch:[300/2000] err:0.1527 alpha:0.1778
epoch:[400/2000] err:0.1529 alpha:0.1785
epoch:[500/2000] err:0.1530 alpha:0.1791
epoch:[600/2000] err:0.1531 alpha:0.1797
epoch:[700/2000] err:0.1530 alpha:0.1802
epoch:[800/2000] err:0.1528 alpha:0.1808
epoch:[900/2000] err:0.1526 alpha:0.1812
epoch:[1000/2000] err:0.1522 alpha:0.1817
epoch:[1100/2000] err:0.1518 alpha:0.1821
epoch:[1200/2000] err:0.1514 alpha:0.1825
epoch:[1300/2000] err:0.1508 alpha:0.1829
epoch:[1400/2000] err:0.1503 alpha:0.1832
epoch:[1500/2000] err:0.1496 alpha:0.1836
epoch:[1600/2000] err:0.1489 alpha:0.1839
epoch:[1700/2000] err:0.1482 alpha:0.1842
epoch:[1800/2000] err:0.1475 alpha:0.1845
epoch:[1900/2000] err:0.1467 alpha:0.1848
epoch:[2000/2000] err:0.1459 alpha:0.1851
Finished Matching!
---------------------------------
Begin finding the embedded space
Done
---------------------------------
unionCom Done!
time: 0.053243160247802734 seconds
---------------------------------
F

In [8]:

for distance_mode in distance_modes:
    print(f"Running with distance mode: {distance_mode}")

    intersect_adni = pd.read_csv("/nfs/dcmb-lgarmire/boweil/EFIGA/DE_label/DE_ADNI_QN.csv", index_col=0)
    intersect_adni['Label'] = intersect_adni['Label'].replace({
        'subtype1': 'EMCI1',
        'subtype2': 'EMCI2'
    })
    EMCI_conditions = intersect_adni['Label'].isin(['EMCI1', 'EMCI2'])
    intersect_adni_EMCI = intersect_adni[EMCI_conditions]
    labels_ADNI = intersect_adni_EMCI['Label']
    intersect_adni_EMCI.drop(columns=['Label'], inplace=True)

    intersect_EFIGA_mci = pd.read_csv("/nfs/dcmb-lgarmire/boweil/EFIGA/DE_label/DE_EFIGA_QN.csv", index_col=0)

    EFIGA_EMCI_samples = intersect_EFIGA_mci.loc[
        intersect_EFIGA_mci['big label'] == 'EMCI'
    ]
    data2_EMCI = EFIGA_EMCI_samples.to_numpy()
    data2_EMCI = data2_EMCI[:, 1:]

    min_max_scaler = MinMaxScaler()
    norm_data1 = min_max_scaler.fit_transform(intersect_adni_EMCI.to_numpy())
    norm_data2 = min_max_scaler.fit_transform(data2_EMCI)

    uc = UnionCom.UnionCom(output_dim=11, epoch_pd=2000, epoch_DNN=0, distance_mode=distance_mode)
    integrated_data = uc.fit_transform(dataset=[norm_data1, norm_data2])
    cor_pairs = uc.match(dataset=[norm_data1, norm_data2])
    norm_pat_match = cor_pairs[0]

    print(f"norm_data1 shape: {norm_data1.shape}")
    print(f"norm_data2 shape: {norm_data2.shape}")
    print(f"norm_pat_match shape: {norm_pat_match.shape}")

    EMCI1_ID = list(np.where(np.isin(labels_ADNI, 'EMCI1'))[0])
    EMCI2_ID = list(np.where(np.isin(labels_ADNI, 'EMCI2'))[0])
    print(f"EMCI1_ID length: {len(EMCI1_ID)}")
    print(f"EMCI2_ID length: {len(EMCI2_ID)}")

    matched_EMCI1, matched_EMCI2 = [], []
    ties = 0

    for index in range(norm_pat_match.shape[0]):
        match_vec = norm_pat_match[index]
        if max(EMCI1_ID) >= len(match_vec) or max(EMCI2_ID) >= len(match_vec):
            continue

        max_score_EMCI1 = max(match_vec[EMCI1_ID])
        max_score_EMCI2 = max(match_vec[EMCI2_ID])

        if abs(max_score_EMCI1 - max_score_EMCI2) < 1e-6:
            ties += 1
            print(f"Tie detected for sample index {index}.")

        if max_score_EMCI1 >= max_score_EMCI2:
            matched_EMCI1.append(index)
        else:
            matched_EMCI2.append(index)

    print(f"Number of matched EMCI1 samples: {len(matched_EMCI1)}")
    print(f"Number of matched EMCI2 samples: {len(matched_EMCI2)}")
    print(f"Number of ties detected: {ties}")

    flat_list_EMCI1 = list(set(sorted(matched_EMCI1)))
    flat_list_EMCI2 = list(set(sorted(matched_EMCI2)))

    intersect_EFIGA_mci['labels_matching_EMCI'] = 'Unmatched'
    efiga_emci_indices = intersect_EFIGA_mci.loc[intersect_EFIGA_mci['big label'] == 'EMCI'].index

    for i, idx in enumerate(efiga_emci_indices):
        if i in flat_list_EMCI1:
            intersect_EFIGA_mci.at[idx, 'labels_matching_EMCI'] = 'EMCI1'
        elif i in flat_list_EMCI2:
            intersect_EFIGA_mci.at[idx, 'labels_matching_EMCI'] = 'EMCI2'

    unmatched_indices = intersect_EFIGA_mci.loc[
        (intersect_EFIGA_mci['big label'] == 'EMCI') & 
        (intersect_EFIGA_mci['labels_matching_EMCI'] == 'Unmatched')
    ].index

    for idx in unmatched_indices:
        col_idx = list(efiga_emci_indices).index(idx)
        similarity_scores = norm_pat_match[:, col_idx]
        highest_similarity_index = np.argmax(similarity_scores)
        highest_similarity_label = labels_ADNI.iloc[highest_similarity_index]
        intersect_EFIGA_mci.at[idx, 'labels_matching_EMCI'] = highest_similarity_label

    final_unmatched_count = intersect_EFIGA_mci.loc[
        (intersect_EFIGA_mci['big label'] == 'EMCI') & 
        (intersect_EFIGA_mci['labels_matching_EMCI'] == 'Unmatched')
    ].shape[0]

    print(f"Remaining unmatched EMCI samples: {final_unmatched_count}")
    print(f"Number of ties detected and resolved: {ties}")

    output_file = f'/nfs/turbo/umms-lgarmire/home/yhdu/Bowei_NAS/EFIGA/Review/Unioncome_Review/EMCI_mis_{distance_mode}.csv'
    intersect_EFIGA_mci.to_csv(output_file)

    print(f"The inferred labels have been saved to '{output_file}'.")


Running with distance mode: geodesic
use random seed: 666
Shape of Raw data
Dataset 0: (201, 11)
Dataset 1: (115, 11)
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
use device: cpu


/tmp/ipykernel_3063453/2865629657.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersect_adni_EMCI.drop(columns=['Label'], inplace=True)


epoch:[100/2000] err:0.3189 alpha:0.5701
epoch:[200/2000] err:0.3144 alpha:0.5636
epoch:[300/2000] err:0.3146 alpha:0.5662
epoch:[400/2000] err:0.3145 alpha:0.5688
epoch:[500/2000] err:0.3142 alpha:0.5714
epoch:[600/2000] err:0.3135 alpha:0.5737
epoch:[700/2000] err:0.3126 alpha:0.5760
epoch:[800/2000] err:0.3114 alpha:0.5781
epoch:[900/2000] err:0.3099 alpha:0.5801
epoch:[1000/2000] err:0.3083 alpha:0.5820
epoch:[1100/2000] err:0.3066 alpha:0.5838
epoch:[1200/2000] err:0.3048 alpha:0.5856
epoch:[1300/2000] err:0.3030 alpha:0.5872
epoch:[1400/2000] err:0.3012 alpha:0.5887
epoch:[1500/2000] err:0.2993 alpha:0.5902
epoch:[1600/2000] err:0.2976 alpha:0.5915
epoch:[1700/2000] err:0.2958 alpha:0.5928
epoch:[1800/2000] err:0.2942 alpha:0.5940
epoch:[1900/2000] err:0.2926 alpha:0.5951
epoch:[2000/2000] err:0.2911 alpha:0.5962
Finished Matching!
---------------------------------
Begin finding the embedded space
Done
---------------------------------
unionCom Done!
time: 0.054849863052368164 se

/tmp/ipykernel_3063453/2865629657.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intersect_adni_EMCI.drop(columns=['Label'], inplace=True)


epoch:[100/2000] err:0.4843 alpha:0.5707
epoch:[200/2000] err:0.4738 alpha:0.5629
epoch:[300/2000] err:0.4711 alpha:0.5659
epoch:[400/2000] err:0.4676 alpha:0.5692
epoch:[500/2000] err:0.4631 alpha:0.5724
epoch:[600/2000] err:0.4580 alpha:0.5755
epoch:[700/2000] err:0.4524 alpha:0.5785
epoch:[800/2000] err:0.4467 alpha:0.5813
epoch:[900/2000] err:0.4412 alpha:0.5840
epoch:[1000/2000] err:0.4360 alpha:0.5865
epoch:[1100/2000] err:0.4312 alpha:0.5888
epoch:[1200/2000] err:0.4270 alpha:0.5909
epoch:[1300/2000] err:0.4232 alpha:0.5928
epoch:[1400/2000] err:0.4199 alpha:0.5945
epoch:[1500/2000] err:0.4170 alpha:0.5961
epoch:[1600/2000] err:0.4144 alpha:0.5975
epoch:[1700/2000] err:0.4121 alpha:0.5988
epoch:[1800/2000] err:0.4100 alpha:0.5999
epoch:[1900/2000] err:0.4082 alpha:0.6010
epoch:[2000/2000] err:0.4064 alpha:0.6020
Finished Matching!
---------------------------------
Begin finding the embedded space
Done
---------------------------------
unionCom Done!
time: 0.05648446083068848 sec